In [0]:
# Performing Joins
import datetime
import pyspark.sql.functions as F
from pyspark.sql import Row
import pandas as pd


courses = [
    {
        'course_id': 1,
        'course_title': 'Python for Data Science',
        'course_published_dt': datetime.date(2020, 1, 1),
        'is_active': True,
        'last_updated_ts': datetime.datetime(2020, 1, 1, 10, 10, 10)
    },
    {
        'course_id': 2,
        'course_title': 'Data Engineering with Apache Spark',
        'course_published_dt': datetime.date(2020, 2, 1),
        'is_active': True,
        'last_updated_ts': datetime.datetime(2020, 2, 1, 10, 10, 10)
    },
    {
        'course_id': 3,
        'course_title': 'Data Science with Python',
        'course_published_dt': datetime.date(2020, 3, 1),
        'is_active': True,
        'last_updated_ts': datetime.datetime(2020, 3, 1, 10, 10, 10)
    },
    {
        'course_id': 4,
        'course_title': 'Data Science with R',
        'course_published_dt': datetime.date(2020, 4, 1),
        'is_active': False,
        'last_updated_ts': datetime.datetime(2020, 4, 1, 10, 10, 10)
    }
]

courses_df = spark.createDataFrame([Row(**course) for course in courses])

In [0]:
users = [
    {
        'user_id': 1,
        'user_first_name': 'John',
        'user_last_name': 'Doe',
        'user_email': 'john.doe@databricks.com'
    },
    {
        'user_id': 2,
        'user_first_name': 'Jane',
        'user_last_name': 'Doe',
        'user_email': 'jane.doe@databricks.com'
    },
    {
        'user_id': 3,
        'user_first_name': 'Alice',
        'user_last_name': 'Williams',
        'user_email': 'alice.williams@databricks.com'
    },
    {
        'user_id': 4,
        'user_first_name': 'Bob',
        'user_last_name': 'Smith',
        'user_email': 'bob.smith@databricks.com'
    }
]

users_df = spark.createDataFrame([Row(**user) for user in users])

In [0]:
course_enrollments = [
    {
        'course_enrollment_id': 1,
        'user_id': 3,
        'course_id': 2,
        'price_paid': 9.99
    },
    {
        'course_enrollment_id': 2,
        'user_id': 4,
        'course_id': 3,
        'price_paid': 19.99
    },
    {
        'course_enrollment_id': 3,
        'user_id': 1,
        'course_id': 1,
        'price_paid': 14.99
    },
    {
        'course_enrollment_id': 4,
        'user_id': 2,
        'course_id': 4,
        'price_paid': 29.99
    }
]

course_enrollments_df = spark.createDataFrame([Row(**course_enrollment) for course_enrollment in course_enrollments])

In [0]:
courses_df.show()

In [0]:
users_df.show()

In [0]:
course_enrollments_df.show()

In [0]:
help(courses_df.join)

In [0]:
help(courses_df.crossJoin)

In [0]:
help(courses_df.alias)

In [0]:
courses_df.alias('c').select('c.course_id').show()

In [0]:
# Inner Joins
users_df \
    .join(course_enrollments_df, users_df.user_id == course_enrollments_df.user_id) \
    .show()

In [0]:
# As both datafrmaes have user_id using the same name, we can pass column name as a string as well
users_df \
    .join(course_enrollments_df, 'user_id') \
    .show()

In [0]:
users_df \
    .join(course_enrollments_df, users_df.user_id == course_enrollments_df.user_id) \
    .select(users_df['*'], course_enrollments_df['course_id'], course_enrollments_df['course_enrollment_id']) \
    .show()

In [0]:
users_df.alias('u') \
    .join(course_enrollments_df.alias('ce'), users_df.user_id == course_enrollments_df.user_id) \
    .select('u.*', 'course_id', 'course_enrollment_id') \
    .show()

In [0]:
users_df \
    .join(course_enrollments_df.alias('ce'), users_df.user_id == course_enrollments_df.user_id) \
    .groupBy(users_df['user_id']) \
    .count() \
    .show()

In [0]:
users_df.alias('u') \
    .join(course_enrollments_df.alias('ce'), users_df.user_id == course_enrollments_df.user_id) \
    .groupBy('u.user_id') \
    .count() \
    .show()

In [0]:
users_df \
    .join(course_enrollments_df, 'user_id') \
    .groupBy('user_id') \
    .count() \
    .show()

In [0]:
# Performing left outer joins using left
users_df \
    .join(course_enrollments_df, users_df.user_id == course_enrollments_df.user_id, 'left') \
    .show()

In [0]:
# left or left_outer or leftouter are the same
users_df \
    .join(course_enrollments_df, users_df.user_id == course_enrollments_df.user_id, 'left_outer') \
    .show()

In [0]:
users_df \
    .join(course_enrollments_df, 'user_id', 'leftouter') \
    .show()

In [0]:
users_df \
    .join(course_enrollments_df, users_df.user_id == course_enrollments_df.user_id, 'left') \
    .select(users_df['*'], course_enrollments_df['course_id'], course_enrollments_df['course_enrollment_id']) \
    .show()

In [0]:
users_df.alias('u') \
    .join(course_enrollments_df.alias('ce'), users_df.user_id == course_enrollments_df.user_id, 'left') \
    .select('u.*', 'course_id', 'course_enrollment_id') \
    .show()

In [0]:
users_df.alias('u') \
    .join(course_enrollments_df.alias('ce'), users_df.user_id == course_enrollments_df.user_id, 'left') \
    .filter('ce.course_enrollment_id IS NULL') \
    .select('u.*', 'course_id', 'course_enrollment_id') \
    .show()

In [0]:
users_df.alias('u') \
    .join(course_enrollments_df.alias('ce'), users_df.user_id == course_enrollments_df.user_id, 'outer') \
    .groupBy('u.user_id') \
    .agg(F.sum(F.when(course_enrollments_df['user_id'].isNull(), 0).otherwise(1)).alias('course_count')) \
    .orderBy('u.user_id') \
    .show()

In [0]:
users_df.alias('u') \
    .join(course_enrollments_df.alias('ce'), users_df.user_id == course_enrollments_df.user_id, 'outer') \
    .groupBy('u.user_id') \
    .agg(F.sum(F.expr('CASE WHEN ce.course_enrollment_id IS NULL THEN 0 ELSE 1 END')).alias('course_count')) \
    .orderBy('u.user_id') \
    .show()

In [0]:
# Right Outer Joins
course_enrollments_df \
    .join(users_df, users_df.user_id == course_enrollments_df.user_id, 'right') \
    .show()

In [0]:
course_enrollments_df \
    .join(users_df, 'user_id', 'right') \
    .show()

In [0]:
course_enrollments_df.alias('ce') \
    .join(users_df.alias('u'), users_df.user_id == course_enrollments_df.user_id, 'right_outer') \
    .select('u.*', 'course_id', 'course_enrollment_id') \
    .show()

In [0]:
# Full Outer Joins
users_df \
    .join(course_enrollments_df, users_df.user_id == course_enrollments_df.user_id, 'full') \
    .show()

In [0]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '0')

In [0]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '10485760b')

In [0]:
# ~1 GB in size
clickstream = spark.read.csv('dbfs:/databricks-datasets/wikipedia-datasets/data-001/clickstream/raw-uncompressed/', sep='\t', header=True)

In [0]:
# 10+ GB in size
articles = spark.read.parquet('dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/')

In [0]:
clickstream.join(articles, articles.id == clickstream.curr_id).count()

In [0]:
%%time

F.broadcast(clickstream).join(articles, articles.id == clickstream.curr_id).count()

In [0]:
help(courses_df.crossJoin)

In [0]:
# Cross Joins
users_df \
    .crossJoin(courses_df) \
    .show()

In [0]:
users_df \
    .join(courses_df) \
    .show()

In [0]:
users_df \
    .join(courses_df, how='cross') \
    .show()